In [72]:
import csv, os, time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

page_num = '1'
i = 1
pro_data = []

with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    driver.get("https://career.programmers.co.kr/job?page={}&order=recent".format(page_num))
    time.sleep(2)
    # 여러 요소를 찾습니다.
    except_elem = driver.find_elements(By.CLASS_NAME, 'external-label')

    # 각 요소에 대해 반복합니다.
    for elem_ex in except_elem:
        # style 속성의 값을 가져옵니다.
        style_attribute = elem_ex.get_attribute('style')

        # style 속성에 display: none;이 포함되어 있는지 확인
        if 'display: none;' in style_attribute:
            # i번째 페이지를 클릭해줍니다.
            button = driver.find_element(By.XPATH, '//*[@id="list-positions-wrapper"]/ul/li[{}]'.format(i))
            ActionChains(driver).click(button).perform()
        
            time.sleep(2)
        
        if len(driver.window_handles) > 1:

            # 새 탭의 핸들을 가져옴
            new_tab_handle = driver.window_handles[1]

            # 새 탭으로 전환
            driver.switch_to.window(new_tab_handle)

            # 새 탭에서 정보를 가져옴
            pro_title = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.TAG_NAME, 'h2')))
            pro_jobs = driver.find_element(By.XPATH,'//*[@id="career-app-legacy"]/div/div[1]/div[1]/section/div/div[1]/div[1]/div[2]')
            pro_stacks = driver.find_elements(By.XPATH, '//*[contains(@class, "QdgvMJO9ZYOaiwrEUqgo") and contains(@class, "nUBs27jXBxRVUu9DLzz4")]')
            
            jobs_list = pro_jobs.text.split(', ')
            
            stacks_list = []
            
            for stack in pro_stacks:
                stacks_list.append(stack.text)
            
            pro_elem = {
                "title" : pro_title.text,
                "jobs" : jobs_list,
                "stacks" : stacks_list
            }
            pro_data.append(pro_elem)
            
            # 새 창이 열렸을 경우, 새 창을 닫음
            driver.close()  # 새 창 닫음
            driver.switch_to.window(driver.window_handles[0])  # 다시 원래 창으로 이동
        i+=1
    
# CSV 파일로 저장
csv_file_path = os.path.join("assets", "data", "pro_info.csv")

# CSV 파일로 저장
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=["title", "jobs", "stacks"])
    writer.writeheader()
    for elem in pro_data:
        writer.writerow(elem)

In [73]:
import csv, os, requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

language = "python"

# assets 폴더가 없으면 생성
folder_name = "assets"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# assets/img/sof 폴더가 없으면 생성
folder_sof = "assets/img/sof"
if not os.path.exists(folder_sof):
    os.makedirs(folder_sof)

with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    driver.get("https://stackoverflow.com/questions/tagged/{}?tab=active&page=1&pagesize=15".format(language))
    
    i = 1
    sof_data = []
    
    # title 경로 찾기
    id_list = []
    
    # 질문 요소들을 찾기 위한 XPath
    id_elements = driver.find_elements(By.XPATH, '//div[contains(@class, "s-post-summary    js-post-summary")]')
    
    # 각 질문 요소에 대해 반복
    for id_element in id_elements:
        # 질문 요소의 id 속성 가져오기
        question_id = id_element.get_attribute("id")
        id_list.append(question_id)
    
    # 인덱스가 리스트의 길이보다 작을 때 반복
    while i <= len(id_list):
        # 요소 찾기
        question_title = driver.find_element(By.XPATH, '//*[@id="{}"]/div[2]/h3/a'.format(id_list[i-1]))
        question_time = driver.find_element(By.XPATH, '//*[@id="{}"]/div[2]/div[2]/div[2]/time/a/span'.format(id_list[i-1]))
        question_writer = driver.find_element(By.XPATH, '//*[@id="{}"]/div[2]/div[2]/div[2]/div/div/a'.format(id_list[i-1]))
        question_img = driver.find_element(By.XPATH, '//*[@id="{}"]/div[2]/div[2]/div[2]/a/div/img'.format(id_list[i-1]))
        
        # 이미지 소스 URL 가져오기
        img_src = question_img.get_attribute("src")
        
        # 이미지 다운로드
        img_name = question_writer.text + ".jpg"  # 이미지 파일 이름 설정
        img_path = os.path.join("assets", "img", "sof", img_name)  # 이미지 파일 경로 설정
        
        # 이미지 다운로드 및 저장
        with open(img_path, "wb") as img_file:
            img_file.write(requests.get(img_src).content)
        
        # 요소의 title 속성값 가져와 sof_data에 저장
        sof_elem = {
            "title" : question_title.text,
            "time" : question_time.text,
            "writer" : question_writer.text,
        }
        sof_data.append(sof_elem)
        i+=1

# CSV 파일로 저장
csv_file_path = os.path.join("assets", "data", "sof_info.csv")

# CSV 파일로 저장
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=["title", "time", "writer"])
    writer.writeheader()
    for elem in sof_data:
        writer.writerow(elem)

In [83]:
import re, csv, os, requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains

# repo_info의 값에서 사용자는 제외하고 뒤의 제목만 가져오기 위해 re라이브러리를 사용했습니다.
def extract_name(repository):
    match = re.match(r'.+?/([^/]+)$', repository)
    if match:
        return match.group(1)
    return None

'''
※필독!※
language와 sort_tag를 정보에 맞게 넣어주세요 language = 개발언어, sort_tag = 'stars' or 'forks'
로그인 상태를 확인해주세요 아이디와 비밀번호를 자신의 비밀번호와 아이디로 넣어주세요
'''

language = "개발언어"
sort_tag = "stars"
your_id = "아이디"
your_password = "비밀번호"

with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    driver.get("https://github.com/topics/{}?o=desc&s={}".format(language, sort_tag))
    
    repositorys = driver.find_elements(By.CLASS_NAME, "f3 color-fg-muted text-normal lh-condensed")
    
    # 레포지토리 제목을 담을 리스트
    repo_info = []
    
    # 레포지토리 제목을 포함하는 태그 찾기
    repo_list = driver.find_elements(By.TAG_NAME, 'h3')
    
    # 태그에서 텍스트 가져오기
    for link in repo_list:
        repo_info.append(link.text)
    
    # 각 요소에서 공백을 없앤 새로운 리스트 생성
    repo_git_list = [item.replace(" ", "") for item in repo_info]

git_data = []

# assets 폴더가 없으면 생성
folder_name = "assets"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# assets/img/git 폴더가 없으면 생성
folder_git = "assets/img/git"
if not os.path.exists(folder_git):
    os.makedirs(folder_git)

for repo_list in repo_git_list[:-1]:
    # WebDriver 초기화,
    with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
        driver.get("https://github.com/" + repo_list)
        
        driver.implicitly_wait(5)
        # 페이지의 HTML 소스를 가져옵니다.
        html_source = driver.page_source

        # 로그인 상태를 나타내는 클래스명을 확인합니다.
        if 'logged-out' in html_source:
            # 내비게이션 바에서 "로그인" 버튼을 찾아 눌러봅시다.
            button = driver.find_element(By.XPATH, "/html/body/div[1]/div[1]/header/div/div[2]/div/div/div/a")
            ActionChains(driver).click(button).perform()
        
            # "아이디" input 요소에 여러분의 아이디를 입력합니다.
            id_input = driver.find_element(By.XPATH, '//*[@id="login_field"]')
            ActionChains(driver).send_keys_to_element(id_input, your_id).perform()
        
            # "패스워드" input 요소에 여러분의 비밀번호를 입력합니다.
            pw_input = driver.find_element(By.XPATH, '//*[@id="password"]')
            ActionChains(driver).send_keys_to_element(pw_input, your_password).perform()
        
            # "로그인" 버튼을 눌러서 로그인을 완료합니다.
            login_button = driver.find_element(By.XPATH, '//*[@id="login"]/div[4]/form/div/input[13]')
            ActionChains(driver).click(login_button).perform()
        
        driver.implicitly_wait(5)
        
        # 요소 찾기
        repo_watch = driver.find_element(By.ID, 'repo-notifications-counter')
        repo_fork = driver.find_element(By.ID, 'repo-network-counter')
        repo_stars = driver.find_element(By.ID, 'repo-stars-counter-star')
        repo_commit_before = driver.find_element(By.XPATH, '/html/body/div[1]/div[6]/div/main/turbo-frame/div/div/div/div[2]/div[1]/react-partial/div/div/div[3]/div[1]/table/tbody/tr[1]/td/div/div[3]/a/span/span[2]/span')
        
        repo_commit = repo_commit_before.text
        repo_commit_slice = re.split(r'\s', repo_commit)[0]
        
        # 요소를 찾기 위해 find_elements를 사용하여 리스트 형태로 반환합니다.
        recent_t = driver.find_elements(By.TAG_NAME, 'relative-time')
        repo_recent = recent_t[0].get_attribute('title')
        
        # 이미지 요소 찾기
        img_element = driver.find_element(By.XPATH, '/html/body/div[1]/div[6]/div/main/div/div[1]/div[1]/div[1]/img')
        
        # 이미지 소스 URL 가져오기
        img_src = img_element.get_attribute("src")
        
        # 이미지 다운로드
        img_name = extract_name(repo_list) + ".jpg"  # 이미지 파일 이름 설정
        img_path = os.path.join("assets", "img", "git", img_name)  # 이미지 파일 경로 설정
        
        # 이미지 다운로드 및 저장
        with open(img_path, "wb") as img_file:
            img_file.write(requests.get(img_src).content)
        
        # 요소의 title 속성값 가져와 git_data에 저장
        git_elem = {
            "title" : extract_name(repo_list),
            "fork" : repo_fork.get_attribute('title'),
            "stars" : repo_stars.get_attribute('title'),
            "recent_time" : repo_recent,
            "number_of_commit" : repo_commit_slice,
        }
        git_data.append(git_elem)

# CSV 파일로 저장
csv_file_path = os.path.join("assets", "data", "git_info_{}.csv".format(sort_tag))

# CSV 파일로 저장
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=["title", "fork", "stars", "recent_time", "number_of_commit"])
    writer.writeheader()
    for elem in git_data:
        writer.writerow(elem)